# Regressão logística

## Aplicação Regressão Logística

Para aplicar a regressão logística com pesos num quadro de dados, normalmente segue estes passos:

1. Preparar os dados: Certifique-se de que o dataframe esteja limpo e pré-processado. Isso inclui o tratamento de valores ausentes, a codificação de variáveis categóricas e a normalização/padronização dos dados, se necessário.

2. Atribuir pesos: Adicione uma coluna ao seu dataframe que contenha os pesos para cada amostra. Essa coluna será usada para aplicar pesos durante a regressão logística.

3. Escolha uma implementação de regressão logística: Use uma biblioteca como scikit-learn, statsmodels ou outra biblioteca adequada que suporte regressão logística ponderada

> Você pode calcular pesos de amostra baseado em alguma lógica que faça sentido para o seu problema. Um método comum é usar o inverso da frequência das classes para balancear o dataset, especialmente se você tiver um problema de desbalanceamento de classes.

In [1]:
# importar bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_sample_weight

In [2]:
# Carregar dataframe
dataframe = pd.read_csv('df_atualizada_2.csv')

# Preparar as variáveis independentes e dependentes
dff = dataframe.drop('CAUSABAS', axis=1)
X = dff.drop('suicidio', axis=1)
y = dff['suicidio']

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Calcular pesos das amostras com base na distribuição das classes
sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

# Treinar o modelo de regressão logística com pesos
model = LogisticRegression()
model.fit(X_train, y_train, sample_weight=sample_weights)

# Fazer previsões com o conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.7374431847198816
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.74      0.85    155159
           1       0.03      0.74      0.05      1489

    accuracy                           0.74    156648
   macro avg       0.51      0.74      0.45    156648
weighted avg       0.99      0.74      0.84    156648



## Iterações

O aviso de convergência que você está vendo sugere que o otimizador não conseguiu convergir para uma solução dentro do número padrão de iterações. Existem algumas abordagens que você pode tomar para resolver esse problema:

1. Aumentar o número de iterações (max_iter):
Você pode aumentar o número de iterações para permitir que o otimizador tenha mais tempo para convergir.

2. Escalar os dados:
Escalar os dados pode ajudar o otimizador a convergir mais rapidamente. Isso é especialmente importante para algoritmos de aprendizado de máquina baseados em gradiente, como a regressão logística.

3. Experimentar diferentes solvers:
O scikit-learn oferece vários solvers que você pode experimentar. Alguns solvers podem convergir melhor para certos tipos de dados.

In [3]:
#importar bibliotecas
from sklearn.preprocessing import StandardScaler

# Escalar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Treinar o modelo de regressão logística com pesos
model = LogisticRegression(class_weight='balanced', max_iter=1000, solver='saga')
model.fit(X_train, y_train, sample_weight=sample_weights)

# Fazer previsões com o conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.13975920535212707
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.13      0.23    155159
           1       0.01      0.97      0.02      1489

    accuracy                           0.14    156648
   macro avg       0.50      0.55      0.13    156648
weighted avg       0.99      0.14      0.23    156648



/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## Aplicando SMOTE

`SMOTE` *(Synthetic Minority Over-sampling Technique)* é uma técnica de oversampling usada para tratar problemas de desbalanceamento de classes em conjuntos de dados. Em problemas de classificação, quando uma classe é significativamente menos representada em comparação com outras (classe minoritária), o modelo pode ter dificuldade em aprender os padrões dessa classe, resultando em desempenho insatisfatório.

### Como Funciona o SMOTE:

1. Seleção dos Vizinhos:

Para cada exemplo da classe minoritária, o SMOTE seleciona seus k vizinhos mais próximos usando uma métrica de distância, como a distância Euclidiana.


2. Geração de Exemplos Sintéticos:

Novos exemplos sintéticos são gerados ao longo das linhas que conectam os exemplos minoritários selecionados aos seus vizinhos.

A diferença entre o exemplo minoritário original e o seu vizinho mais próximo é multiplicada por um número aleatório entre 0 e 1, e essa diferença é adicionada ao exemplo original, criando um novo ponto de dados.


### Vantagens do SMOTE:

1. Aumento da Variabilidade:

Ao criar exemplos sintéticos, o SMOTE aumenta a variabilidade da classe minoritária, ajudando o modelo a generalizar melhor.

2. Prevenção de Overfitting:

Diferente do simples duplicação de exemplos minoritários, o SMOTE gera novos exemplos únicos, ajudando a prevenir o overfitting.

### Desvantagens do SMOTE:

1. Introdução de Ruído:

Exemplos sintéticos podem estar em regiões do espaço de características onde a classe minoritária não deveria estar presente, introduzindo ruído.


2. Computacionalmente Intenso:

Para grandes conjuntos de dados, a geração de exemplos sintéticos pode ser computacionalmente intensiva.

3. Implementação do SMOTE em Python:

A biblioteca imblearn (Imbalanced-learn) oferece uma implementação conveniente do SMOTE. 

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.over_sampling import SMOTE

# Carregar dataframe
dataframe = pd.read_csv('df_atualizada_2.csv')

# Preparar as variáveis independentes e dependentes
dff = dataframe.drop('CAUSABAS', axis=1)
X = dff.drop('suicidio', axis=1)
y = dff['suicidio']

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Aplicar SMOTE para oversampling
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Treinar o modelo de regressão logística com os dados balanceados
model = LogisticRegression(max_iter=1000, solver='saga')
model.fit(X_train_res, y_train_res)

# Fazer previsões com o conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7646442980440222
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.76      0.87    155159
           1       0.03      0.78      0.06      1489

    accuracy                           0.76    156648
   macro avg       0.51      0.77      0.46    156648
weighted avg       0.99      0.76      0.86    156648



### Observação

Os resultados indicam que, apesar de ter utilizado SMOTE, o modelo ainda possui uma precisão muito baixa para a classe minoritária (1), enquanto o recall é razoavelmente alto. Isso sugere que o modelo está detectando a maioria dos casos da classe minoritária, mas também classificando muitos casos da classe majoritária como minoritária, o que resulta em baixa precisão.

Aqui estão algumas sugestões adicionais para melhorar o desempenho:

1. Ajuste Fino de Hiperparâmetros:

`Você pode ajustar os hiperparâmetros do modelo para encontrar um melhor balanço entre precisão e recall.`

2. Métodos Ensemble:

`Random Forests e Gradient Boosting geralmente têm um melhor desempenho em problemas desbalanceados.`

3. Experimentar Outros Solvers e Modelos:

`Tentar outros modelos de regressão logística ou solvers, como liblinear, newton-cg, ou lbfgs.`


### Ajustes de Hiperparâmetros

No código abaixo, estamos: 

1. Aplicando `SMOTE` nos dados de treinamento`

2. Utilizando a grade de ``Hiperparâmetros``, como os parâmetros ``Solve``, ``Iterações`` e ``Pesos de classes``, a fim de validação

3. E realizando uma `Validação Cruzada` de modelos de `Regressão logística`, buscando identificar quais os melhores `PARÂMETROS`, da grade de Hiperparâmetros, e modelos de regressão

4. Treinando o modelo com os melhores `PARÂMETROS` encontrados

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Carregar o dataframe
dataframe = pd.read_csv('df_atualizada_2.csv')

# Preparar as variáveis independentes e dependentes
dff = dataframe.drop('CAUSABAS', axis=1)
X = dff.drop('suicidio', axis=1)
y = dff['suicidio']

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Aplicar SMOTE para oversampling
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Definir o grid de hiperparâmetros
param_grid = {
    'solver': ['lbfgs', 'liblinear', 'saga'],
    'max_iter': [1000, 3000, 5000],
    'class_weight': ['balanced', {0: 1, 1: 50}, {0: 1, 1: 100}]
}

# Realizar a busca em grade com validação cruzada
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='recall')
grid.fit(X_train_res, y_train_res)

# Imprimir os melhores parâmetros encontrados
print("Best parameters:", grid.best_params_)
print("Best score:", grid.best_score_)

# Treinar o modelo com os melhores parâmetros encontrados
best_model = grid.best_estimator_
best_model.fit(X_train_res, y_train_res)

# Fazer previsões com o conjunto de teste
y_pred = best_model.predict(X_test)

# Avaliar o modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max

Best parameters: {'class_weight': {0: 1, 1: 100}, 'max_iter': 1000, 'solver': 'lbfgs'}
Best score: 0.9997841995517156
Accuracy: 0.030131249680813033
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.02      0.04    155159
           1       0.01      1.00      0.02      1489

    accuracy                           0.03    156648
   macro avg       0.50      0.51      0.03    156648
weighted avg       0.99      0.03      0.04    156648



### Manejo de Classes Desbalanceadas

Problemas desbalanceados referem-se a situações em que uma classe (ou várias) na sua base de dados de treinamento têm muito menos exemplos do que outras. Isso é comum em problemas como detecção de fraudes, diagnósticos médicos raros, entre outros. Métodos como Random Forests e Gradient Boosting tendem a lidar melhor com isso porque:

`Random Forests`: Por construir múltiplas árvores de decisão e combinar seus resultados, ele pode gerar modelos robustos que mitigam o impacto do desbalanceamento ao considerar diferentes subconjuntos dos dados.

`Gradient Boosting`: Ao treinar iterativamente em cima dos erros dos modelos anteriores, Gradient Boosting foca nos exemplos mais difíceis de classificar, o que pode melhorar o desempenho nas classes minoritárias.

- Embora esses métodos sejam eficazes, ajustes de parâmetros específicos (como pesos de classe ou taxas de aprendizado) ainda podem ser necessários para otimizar o desempenho em problemas desbalanceados. Isso pode incluir técnicas como oversampling (aumentar exemplos da classe minoritária) ou undersampling (reduzir exemplos da classe majoritária).

- Enquanto Random Forests e Gradient Boosting são frequentemente recomendados, outros métodos como SVMs com kernels adequados ou redes neurais também podem ser eficazes dependendo do contexto e do conjunto de dados específico.


#### Random Forest

**Random Forest** é um algoritmo de aprendizado de máquina baseado em árvores de decisão. Ele cria uma "floresta" de árvores de decisão durante o treinamento e faz previsões agregando as previsões das árvores individuais. Esse método ajuda a reduzir o overfitting e melhora a precisão do modelo.

##### Como funciona:

1. **Ensemble de Árvores de Decisão**: Cria várias árvores de decisão, cada uma treinada com um subconjunto diferente dos dados.
2. **Bagging (Bootstrap Aggregating)**: Utiliza amostragem com reposição para criar subconjuntos de dados para cada árvore.
3. **Agregação**: A previsão final é feita agregando (por exemplo, votando ou tirando a média) as previsões de todas as árvores.

##### Implementação:

Vamos usar a biblioteca `scikit-learn` para treinar um modelo Random Forest. Utilizaremos a técnica SMOTE para balancear as classes.

```python
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Carregar o dataframe
dataframe = pd.read_csv('df_atualizada_2.csv')

# Preparar as variáveis independentes e dependentes
dff = dataframe.drop('CAUSABAS', axis=1)
X = dff.drop('suicidio', axis=1)
y = dff['suicidio']

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Aplicar SMOTE para oversampling
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Treinar o modelo Random Forest com pesos balanceados
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_model.fit(X_train_res, y_train_res)

# Fazer previsões com o conjunto de teste
y_pred = rf_model.predict(X_test)

# Avaliar o modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
```

#### Gradient Boosting

**Gradient Boosting** é um algoritmo de aprendizado de máquina que cria uma série de árvores de decisão, onde cada árvore corrige os erros do modelo anterior. É especialmente eficaz em conjuntos de dados desbalanceados porque pode se concentrar nos exemplos mal classificados.

##### Como funciona:

1. **Inicialização**: Começa com uma árvore de decisão simples.
2. **Correção de Erros**: Cada árvore subsequente é treinada para corrigir os erros residuais das árvores anteriores.
3. **Combinação de Árvores**: As árvores são combinadas para fazer a previsão final, geralmente através de uma média ponderada.

##### Implementação:

Vamos usar a biblioteca `scikit-learn` para treinar um modelo Gradient Boosting. Utilizaremos a técnica SMOTE para balancear as classes.

```python
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Carregar o dataframe
dataframe = pd.read_csv('df_atualizada_2.csv')

# Preparar as variáveis independentes e dependentes
dff = dataframe.drop('CAUSABAS', axis=1)
X = dff.drop('suicidio', axis=1)
y = dff['suicidio']

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Aplicar SMOTE para oversampling
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Treinar o modelo Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train_res, y_train_res)

# Fazer previsões com o conjunto de teste
y_pred = gb_model.predict(X_test)

# Avaliar o modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
```

#### Explicação das Etapas:

1. **Carregar e Preparar os Dados**:
   - Carregamos os dados e preparamos as variáveis independentes (X) e dependentes (y).
2. **Divisão do Conjunto de Dados**:
   - Dividimos os dados em conjuntos de treinamento e teste.
3. **Escalar os Dados**:
   - Normalizamos os dados para garantir que todas as características estejam na mesma escala.
4. **Aplicar SMOTE**:
   - Usamos o SMOTE para balancear as classes no conjunto de treinamento.
5. **Treinar o Modelo**:
   - Treinamos o modelo Random Forest ou Gradient Boosting com os dados balanceados.
6. **Avaliar o Modelo**:
   - Avaliamos o desempenho do modelo usando métricas como acurácia, precisão, recall e f1-score.

Essas abordagens podem ajudar a melhorar o desempenho do seu modelo em conjuntos de dados desbalanceados. Tente executar esses códigos e observe as mudanças nas métricas de avaliação.

### Aplicando Gradient Boosting

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Carregar o dataframe
dataframe = pd.read_csv('df_atualizada_2.csv')

# Preparar as variáveis independentes e dependentes
dff = dataframe.drop('CAUSABAS', axis=1)
X = dff.drop('suicidio', axis=1)
y = dff['suicidio']

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Aplicar SMOTE para oversampling
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Treinar o modelo Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train_res, y_train_res)

# Fazer previsões com o conjunto de teste
y_pred = gb_model.predict(X_test)

# Avaliar o modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8325353659159389
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.83      0.91    155159
           1       0.04      0.72      0.08      1489

    accuracy                           0.83    156648
   macro avg       0.52      0.78      0.49    156648
weighted avg       0.99      0.83      0.90    156648



"\n\n# definindo variáveis para as variáveis dependentes e independentes (target e data)\nfrom sklearn import tree\n\n# carregando dataframe para modelo de previsao\ndataframe = pd.read_csv('df_atualizada_2.csv')\n\ndff = dataframe.drop('CAUSABAS', axis=1)\n\n# definindo as variaveis dependente (y) e independentes (X)\nX = dff.drop('suicidio', axis=1) # novo dataframe de variáveis independentes e retirar variável dependente ('suicidio' -> axis = eixo das colunas)\ny = dff['suicidio']\n\n# Dividir o conjunto de dados em treinamento e teste\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# treinar classificador\nclf = tree.DecisionTreeClassifier(random_state=42)\nclf = clf.fit(X_train, y_train)\n\n# Fazer previsões com o conjunto de teste\ny_pred = clf.predict(X_test)\n\n"